In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.8 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

train_data = pd.read_csv('/content/drive/MyDrive/datasets/subtask_2/es/train.tsv',sep='\t')
train_data = train_data.reset_index(drop=True)
print(train_data.head())

      id                                               text label
0  12786  Sin embargo, los jóvenes son capaces de recono...     B
1  12361  ¿Hay algo más que quieras compartir? ¿Algo sob...     B
2   1662  El servicio de sala es bueno, rápido y amabilí...     B
3  14729  Para concentrarse en el hablante, trata de des...     F
4   9312  Los responsables locales tendrán ahora que esp...     F


In [5]:
from sklearn.model_selection import train_test_split
train_data_texts = train_data['text'].to_list()
train_data_labels = train_data['label'].to_list()
print("Unique labels: ", set(train_data_labels))


Unique labels:  {'B', 'A', 'F', 'E', 'C', 'D'}


In [6]:
train_texts, test_texts, train_labels, test_labels = train_test_split(train_data_texts, train_data_labels, test_size=0.1)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.1)
print('train data size: ', len(train_texts))
print('validation data size: ', len(val_texts))
print('test data size: ', len(test_texts))

train data size:  17766
validation data size:  1975
test data size:  2194


In [7]:
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

In [8]:
import nltk
# feature augmentation using pos tags
nltk.download('averaged_perceptron_tagger')
# nltk.download('punkt')
tagger = nltk.pos_tag

train_texts_pos = []
for text in train_texts:
    # Tokenize text and get POS tags
    tokens = tokenizer.tokenize(text)
    train_pos_tags = [tag for token, tag in nltk.pos_tag(tokens)]
    train_texts_pos.append(train_pos_tags)

val_texts_pos = []
for text in val_texts:
    # Tokenize text and get POS tags
    tokens = tokenizer.tokenize(text)
    val_pos_tags = [tag for token, tag in nltk.pos_tag(tokens)]
    val_texts_pos.append(val_pos_tags)

test_texts_pos = []
for text in test_texts:
    # Tokenize text and get POS tags
    tokens = tokenizer.tokenize(text)
    test_pos_tags = [tag for token, tag in nltk.pos_tag(tokens)]
    test_texts_pos.append(test_pos_tags)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [10]:
# Convert to a list of strings
train_texts_pos = [' '.join(pos_tags) for pos_tags in train_texts_pos]
val_texts_pos = [' '.join(pos_tags) for pos_tags in val_texts_pos]
test_texts_pos = [' '.join(pos_tags) for pos_tags in test_texts_pos]

train_encodings = tokenizer(train_texts, train_texts_pos, truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(val_texts, val_texts_pos, truncation=True, padding=True, max_length=256)
test_encodings = tokenizer(test_texts, test_texts_pos, truncation=True, padding=True, max_length=256)

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels[idx] == 'A':
            item['labels'] = torch.tensor(0)
        elif self.labels[idx] == 'B':
            item['labels'] = torch.tensor(1)
        elif self.labels[idx] == 'C':
            item['labels'] = torch.tensor(2)
        elif self.labels[idx] == 'D':
            item['labels'] = torch.tensor(3)
        elif self.labels[idx] == 'E':
            item['labels'] = torch.tensor(4)
        elif self.labels[idx] == 'F':
            item['labels'] = torch.tensor(5)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)
test_dataset = CustomDataset(test_encodings, test_labels)

print("Sample Data Point")
print()
print(train_dataset[0])

Sample Data Point

{'input_ids': tensor([    4,  1036,  2522,  1065,  3807,  1440,  1042,   978,  1008,  2426,
         2193,  1009,  1872,  1030,  3352,  2682,   972,  6532,   972,  1799,
         1072,  2968,  2980,  1042,  1072,  1694,  1017,  1008,  2329,  1008,
         2766,  1008,  2682,  1017,  3607,  1013,  1030,  9101, 12432,  1008,
         1065,  7263,  1017, 21392,  1108,  6212,  2197,  1110,  1089,  3749,
         6059,  1008,  4941,  1030,  4042,  1008,  7263,  1051,  3592,  1008,
        19373,  1042, 29592, 23152,  1017,  1038, 30207, 30931,  1049, 14627,
        17472,  1017,  1506,  1184,  1030,  7848,  1074, 12655,  1008,  6502,
         3592,  1009,  1872,  1030,  2235,  3352,  1512,  1096,  3503,  3952,
         3161,  6233,  4461,  1013,  6416,  4909,  7503,  1042,  6156,     5,
         4332,   993,   992,  1083, 30969,   993,   992,  7877,   993,   992,
         7877,  4332,  1083, 30969,  1083, 30969, 30966,  1009,  1083, 30969,
        30966,  1083, 30969,  1

In [11]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='macro')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [12]:
model = AutoModelForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-cased",num_labels=6).to(device)
print("Model Configurations")
print()
print(model.config)

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchi

Model Configurations

BertConfig {
  "_name_or_path": "dccuchile/bert-base-spanish-wwm-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.28.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31002
}



In [13]:
training_args = TrainingArguments(
    output_dir='./results',       
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    evaluation_strategy='epoch',
    save_strategy='epoch',
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.024300,1.094329,0.508861,0.443709,0.541539,0.499541
2,0.754800,0.827221,0.614684,0.596245,0.606916,0.602784
3,0.521400,0.929111,0.615696,0.602209,0.615322,0.605512


TrainOutput(global_step=1668, training_loss=0.8798362271105357, metrics={'train_runtime': 2293.8355, 'train_samples_per_second': 23.235, 'train_steps_per_second': 0.727, 'total_flos': 7011898333267968.0, 'train_loss': 0.8798362271105357, 'epoch': 3.0})

In [14]:
trainer.evaluate(test_dataset)
predictions, labels, _ = trainer.predict(test_dataset)
predictions = np.argmax(predictions, axis=1)

from sklearn.metrics import classification_report
print(classification_report(labels, predictions))

              precision    recall  f1-score   support

           0       0.77      0.62      0.69       372
           1       0.40      0.32      0.36       364
           2       0.46      0.64      0.54       355
           3       0.68      0.44      0.53       349
           4       0.52      0.65      0.58       373
           5       0.81      0.92      0.86       381

    accuracy                           0.60      2194
   macro avg       0.61      0.60      0.59      2194
weighted avg       0.61      0.60      0.60      2194

